In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("fake_or_real_news.csv",encoding="ISO-8859-1")

In [3]:
df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillaryâs Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"â Kaydee King (@KaydeeKing) November 9, 2016...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The âPâ in PBS Should Stand for âPlutocr...,The âPâ in PBS Should Stand for âPlutocr...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia âPresident Obama conve...",REAL


In [4]:
df = df[["text", 'label']]

In [5]:
df

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"â Kaydee King (@KaydeeKing) November 9, 2016...",FAKE
4,It's primary day in New York and front-runners...,REAL
...,...,...
6330,The State Department told the Republican Natio...,REAL
6331,The âPâ in PBS Should Stand for âPlutocr...,FAKE
6332,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,"ADDIS ABABA, Ethiopia âPresident Obama conve...",REAL


In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

In [7]:
import nltk 
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [8]:
class TextNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, y=None, **fit_params):
        X_copy = X.copy()
        for i in range(len(X)):
            X_copy[i] = " ".join([token.lower()
                                   for token in word_tokenize(X_copy[i])])
        return X_copy

In [9]:
norm = TextNormalizer()

In [10]:
norm.fit_transform(["An apple a day, keeps the doctor away!"])

['an apple a day , keeps the doctor away !']

In [11]:
class WordExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, stop_words):
        self.stop_words = stop_words
    
    def fit(self, X, y=None, **fit_params):
        self.general_freq = FreqDist()
        for document in X:
            tokens = word_tokenize(document)
            freq = FreqDist(tokens)
            self.general_freq.update(freq)
        self.hapaxes = self.general_freq.hapaxes()
        return self
    
    def transform(self, X, y=None, **fit_params):
        X_copy = X.copy()
        for i in range(len(X)):
            X_copy[i] = ' '.join([token for token in word_tokenize(X[i])
                                 if token not in self.hapaxes and
                                 token not in self.stop_words])
        return X_copy

In [12]:
stop_words= stopwords.words('english')

In [13]:
word_extractor = WordExtractor(stop_words) 

In [14]:
corpus = [
    
    'John is a preaty boy',
    'Ann likes John',
    'Ann likes cherry',
    'Cherry is red'
]

In [15]:
word_extractor.fit_transform(corpus)

['John', 'Ann likes John', 'Ann likes', '']

In [16]:
class ApplyStemmer(BaseEstimator, TransformerMixin):
    def __init__(self, stemmer):
        self.stemmer = stemmer
    
    def fit(self, X, y=None, **fit_tranform):
        return self
    
    def transform(self, X, y=None, **fit_tranform):
        X_copy = X.copy()
        for i in range(len(X)):
            X_copy[i] = ' '.join([self.stemmer.stem(token) 
                                  for token in word_tokenize(X_copy[i])])
        return X_copy

In [17]:
porter_stemmer = PorterStemmer()

In [18]:
apply_stemmer = ApplyStemmer(porter_stemmer)

In [19]:
apply_stemmer.fit_transform(["An apple a day, keeps the doctor away!"])

['an appl a day , keep the doctor away !']

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
pipe = Pipeline([
    ("norm", TextNormalizer()),
    ("extractor", WordExtractor(stop_words)),
    ("stemmer", ApplyStemmer(PorterStemmer())),
    ("vectorizer", CountVectorizer()),
    ("logic", LogisticRegression())
    
])

In [23]:
X = df['text'].values
y = df['label'].values

In [24]:
X_train, X_text, y_train, y_text = train_test_split(X, y)

In [25]:
pipe.fit(X_train, y_train)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('norm', TextNormalizer()),
                ('extractor',
                 WordExtractor(stop_words=['i', 'me', 'my', 'myself', 'we',
                                           'our', 'ours', 'ourselves', 'you',
                                           "you're", "you've", "you'll",
                                           "you'd", 'your', 'yours', 'yourself',
                                           'yourselves', 'he', 'him', 'his',
                                           'himself', 'she', "she's", 'her',
                                           'hers', 'herself', 'it', "it's",
                                           'its', 'itself', ...])),
                ('stemmer', ApplyStemmer(stemmer=<PorterStemmer>)),
                ('vectorizer', CountVectorizer()),
                ('logic', LogisticRegression())])

In [26]:
y_pred = pipe.predict(X_text)

In [27]:
accuracy_score(y_pred, y_text)

0.9122474747474747

In [28]:
import pickle

In [29]:
pickle.dump(pipe, open("nlp_pipe.pkl", 'wb'))